<span type="title">内部类</span> | <span type="update">2018-07-22</span> - Version <span type="version">1.4</span>
    
    
<span type="intro"><p class="card-text">本章主要讲解内部类。对某个标准（接口、抽象类和普通类）的实现（实现或者继承）的类，它外部嵌套了一个类，称这个内部被嵌套的类为内部类。借助于内部类对于外部类的引用和内部类自身权限设置，内部类可用于隐藏对于其实现的接口的具体细节，并且能够操纵外部类资源，这可以提供用户以清晰的内外部逻辑关系，其常用来模拟实现C++中的多重继承(因为Java没有多重继承，所以只有搞这种让别的类去继承，然后把这些类含在自己内部的操作)。</p><p class="card-text">内部类是一种新型的多个类之间的关系，什么时候使用内部类？当单体继承或者接口模式不足以填满标准和外围类之间的隔阂，必须采用一个过度的类以粘合标准和外围的情况下，使用内部类。具体而言，当需要多个对于标准的不同实现、需要处理“一个大的带多个由同一个标准产生的小的”这种控制器模式或者处理“直接继承或者实现逻辑太牵强”这种情况多使用内部类。</p><p class="card-text">对于本章而言，首先介绍了内部类的定义、类型和调用的语法细节，接着介绍了内部类的必要性。在第二部分介绍了不同位置的内部类、不同种类的内部类（匿名内部类和嵌套类）的定义、作用域和适用情况。最后，给出内部类进行设计的一个范例：控制器范式。同时探讨了内部类在闭包中的用途、内部类的继承、局部内部类、内部类标识符等琐碎信息。</p></span>

# 内部类概要

## 内部类的定义、类型和调用

内部类是一种位于类内部的类。其允许将逻辑相关的代码放在一个“块”中，因此可以提高程序的清晰性。内部类不是“组合”，前者提供了一些类方法的实现，而“组合”则重在实例化现有的类，尽管从表面看来，它们在一个类中都持有其他的类。

内部类常常被用来**隐藏代码**，隐藏对于一个接口的具体实现过程。其可以和外围类进行通信，随意调用外围类的私有域和方法，即便声称了 `private`，但是外围类只有在实例化内部类之后才能访问内部类的动态域和方法。


```java
/Sequence.java
import java.util.Random; import static com.mazhangjing.Print.*;
/**
 * 选择器，包括三个方法 end() 是否结束；current()当前对象；next() 计数器加1，不返回值 */
interface Selector {
    boolean end();
    Object current();
    void next();
}
/**
 * 一个定常的序列类，数据保存在 sequence 数组中，next 表示当前序列长度
 * 私有类 sequenceselecter 实现了 selector，在内部操纵计数器 i，完成了判断是否结束、取回当前序列
 * 对应位置对象和计数器加1的操作。
 * */
public class Sequence {
    private Object[] sequence;
    private int next = 0;

    public Sequence(int size){
        sequence = new Object[size];
    }

    public void add(Object n){
        if (next < sequence.length){
            sequence[next++] = n; }
    }
    /**
     * 内部的 SenquenceSelecter 类可以自动获取外部类所有成员权限，不论是否私有。
     * 外部类需要一个方法来传递构造器构造好的内部类对象，在这里使用外部对象 getSelector()，或者
     * 使用 x.new SequenceSelecter() 创建，注意，不能写成 new Sequence.SequenceSelecter();
     * 外部类总是先于内部类进行对象的创建（其中的愿意那时内部类总要引用其所需要的
     * 外部类域或者方法）*/
    private class SequenceSelecter implements Selector {
        private int i = 0;
        public boolean end() {
            return i == sequence.length; }
        public Object current() {
            return sequence[i]; }
        public void next() {
            if (i < sequence.length) {i++;} }
        //对外部类的引用，返回值写外部类名，具体调用写类名.this，返回的是外部对象。
        public Sequence getFather(){
            return Sequence.this;
        }
    }
    //用以说明内部类的接口比外部类更加灵活的例子——可以对相同接口不同的实现
    private class reverseSelecter implements Selector {
        private int i = sequence.length;
        public boolean end() {
            return i == 0; }
        public Object current() {
            return sequence[i-1]; }
        public void next() {
            if (i > 0) {i--;} }
    }

    public Selector getSelector(){ return new SequenceSelecter();}

    public static void main(String[] args){
        Sequence x = new Sequence(10);
        CM2 a = new CM2("Hello World");
        CM2 b = new CM2("Hello CORKINE");
        for (int i = 0; i < 8; i++){ x.add(Integer.toString(i)); }
        x.add(a);
        x.add(b);
        //Selector s = x.getSelector(); 使用外部类方法创建内部类实例
        //需要注意，这里因为是一个接口，所以写成了Selector，如果不想向上转型，默认写作
        //Sequence.SequenceSelecter s = x.getSelector();
        //或者写成下面这样，使用.new直接创建内部类实例
        //因为外部类必须先存在，因此使用 x.new 而不是 new Sequence.SequenceSelecter()
        Sequence.SequenceSelecter s = x.new SequenceSelecter();
        Selector s2 = x.new reverseSelecter();
        while (!s2.end()){
            print(s2.current());
            s2.next();
        }
    }
}
class CM2 {
    private String s;
    CM2(String s){ this.s = s; }
    public String toString() { return this.s; }
}
```

在这个例子中，内部类 `SequenceSelector` 是对于接口 `Selector` 的一种实现。而和之前不同，没有使用外围类 `Sequence` 自身 `implement` 这个接口的原因是，其自身的实现很蹩脚，一个序列实现一个计数控制器从逻辑上来讲很别扭。但是在内部类之前，我们只有这一种选择。

**内部类的结构特征和初始化顺序**

注意这里的内部类语法，`private class XXX implement xxx` 和普通类没有什么区别，关键是其放置的位置，在外围类原本放置方法的地方。此外，当内部类进行构造的时候，其**需要外围类先进行构造**，然后才能引用外围类的域和方法。当构造内部类的时候，其其隐含了一个对于外围类的引用 `Sequence.this`，这用来指向外围类实例本身。再次强调，**内部类拥有外围类的一切访问权限，反过来则不成立。**

**内部类的类型、创建内部类实例的注意事项**

调用一个内部类需要注意内部类类型的问题。首先，如果这个内部类没有实现某个接口或者抽象类或者一般类，那么其类型应该是 `ClassFather.ClassInner` 比如这里的 `Sequence.SequenceSelector` 。而如果实现了某个接口，自然可以向上转型为 `Sequence` 类型。

其次，还要注意调用内部类语法问题。一般而言，内部类都会通过一个外围类的特定方法来返回其实例，比如这里的 `getSelector()` 注意这里的类型问题。但是，如果不通过这种方法，则需要通过 `ClassFatherInstance.new ClassInner()` 这种方式来调用。记得之前说过的，因为内部类可以随意调用外部类的域和方法，因此其必须等待外部类实例化之后才能实例化自身。它们之间的关系和继承中的初始化有些相似。因此，对于本例，使用 `x.new SequenctSelector()` 来创建内部类的实例。注意不能写成 `new Sequence.SequenceSelector()` 因为外部类需要初始化，并且内部类需要有一个外部类的对象的引用 `Sequence.this`。

## 内部类的必要性

**为什么外围类不自己去实现接口？ - 逻辑上不合适，内部代码容易混淆**

为什么外围类不自己实现这个 `Selector` 接口呢？可以，并且根据上一章的知识，它可以正确工作，但是，很多时候，让它自己实现一个接口太过于小题大做，并且实现接口的方法容易和其本身内部的方法造成混淆。

按照我们的例子，因为这里的Selector只用服务于 Sequence，它很小，并且最好向外界隐藏实现，作为一个助手即可。对于这种 `一一对应、任劳任怨` 的助手，应该有一种新的方式来处理 `类与类之间的这种奇妙关系`。因此有了内部类，在这种语法中，即有继承的那种域和方法的共享，并且清晰的理顺了逻辑（把和翻页相关的代码完全放在内部类中，提高了程序清晰性，并且因为内部类只能够通过实例访问，因此具有较高的独立性）。

**为什么内部类不继承外围类？或者不使用代理？ - 逻辑上不合适，难以复用代码**

对于传统的 `定义 - 实现`, 不论是通过对接口还是对于类的继承，有时候都不能够很好的映射现实中的问题。比如说在这里，如果不使用内部类，那么我就需要首先写一个 `SequenceSelector` 以对接口进行实现，然后因为 `Sequence` 需要使用这里的 `Selector`，并且是一个典型的 `is-a` 关系，所以就需要让 `Selector` 继承 `Sequence`（这样的话，在Selector内部就有了Sequence的方法，可以传出当前的Sequence内容），这样很别扭。那么还有一种方式是，使用代理，让 `Sequence` 被 `Selector` 所包裹，后者全权操纵前者，但是这样的结构难以重用（因为进行了实例化）。

因此，对于这种类和类之间的关系，使用 继承 并不合适。

**一言以蔽之：什么时候应该使用内部类？**


对于这个例子所表示的**不平等的类和类之间的关系**，使用继承太驴头不对马嘴（虽然可以工作），使用接口通信逻辑上也难以讲通（虽然可以工作），因此，使用内部类，**一个大的，一个（或一堆相似的）小的**，逻辑清晰易懂，虽然手动调用麻烦了点，但是看起来和用起来其实差别不大。比如在这个例子中，我们可以轻而易举的实现反向的控制器，只需要添加一个私有类，让这个私有类继续实现原有的接口即可。

从某种角度来说，可以将内部类看作是**没有实例化的代理模式**。因为其和代理很像，一堆相同标准的小的类被包裹在一个大的类中，大的类被用来代理小的类的行为。

此外，需要对标准进行多个实现的情况下，内部类也是唯一的选择。总的而言：

## 内部类的本质

**内部类就是一个二道贩子，因为Java自身不能实现多重继承，因此对于一堆单体继承/实现的类进行包含就算完成了多重继承。**对于那些接口需要有多种实现的设计模式，内部类是唯一的选择。而平常情况下，使用内部类可以避免单体类或者接口驴头不对马嘴的逻辑，以及很好的完成内外部逻辑的分离。其实——判断是否使用内部类的方式非常简单：**单体或者接口模式不足以填满标准和外围类之间的隔阂，必须采用一个过度的类以粘合标准和外围的情况下，使用内部类。**

# 不同位置的内部类

## 在方法和作用域内的内部类

内部类不仅仅可以放置在外围类的内部，其也可以放置在外围类方法的内部，甚至是在某个作用域的内部，如下列代码：

```java
/FF.java
import java.util.Random; import static com.mazhangjing.Print.*;
public class FF {
    public I methodA(){
        //这个包裹在方法中的类的作用域仅限于此方法（除了这个方法外，在FF中不可
        // 访问classA），当方法调用完之后就被清理。但是因为有进一步的引用，所以
        //构造的对象现在还不会被垃圾回收。
        class classA implements I {
            public void i() {
                print("Implement of interface I in classA"); }
        }
        return new classA();
    }
    public void methodB() {
        Random rand = new Random();
        //被包裹在作用域内的类，出了作用域就不可用
        if (rand.nextInt(10)%2 == 0){
            class classB implements I {
                public void i() {
                    print("Implement of interface I in classB"); }
            }
            classB b = new classB();
            b.i();
        } else {print("You missed classB");} //在这里已经不可用
    }
    private class classC implements I {
        public void i() {
            print("Implement of interface I in classC"); }
    }
    public I methodC() {
        I x = this.new classC();
        return x;
    }
    public static void main(String[] args){
        FF x = new FF();
        x.methodA().i();
        x.methodB();
        x.methodC().i();
        ((FF.classC)(x.methodC())).i();
        //在其内部看不到向下转型的错误，当这句话在别的类中执行会出错，因为private的对象隐藏了接口的实现
        //这就是之前讲过的使用私有声明的内部类来隐藏接口实现
    }
}
interface I {
    void i();
}
```

可以看到，对于方法体内的内部类来说，当方法调用完，这个类就消失？真的吗？当然不是，如果存在指向这个类的引用，垃圾回收器不会管它，但如果像我们这样使用 `x.methodA().i()` 那么使用完后这个类就从内存中消失了。对于作用于内部的类而言，出了作用域也同样不可访问。内部类所在的位置决定了其可以访问的资源，比如，在 `methodA()` 中的这个内部类，其可以直接访问这个方法的参数，但是除了这个参数，外部如果声明一个 private 方法，它就不可访问了。

## 利用内部类构造更加私有的接口实现

此外还需要注意，`classC` 在这里指的是一个正经的内部类。注意这里的 `private` 关键字，这意味着，如果在这个类外部访问这个内部类，可能吗？当然可能，使用其向上转型的类型 I 和方法 methodC，就可以直接访问。但是，不能使用 `FF.classC` 这种方法。这意味着什么呢？这意味着对于接口的实现，使用内部类的方法做到了完全的屏蔽，对于用户而言，其只知道交互的是一个 `I` 类型，而完全不知道是谁提供的实现，以及提供的实现是什么。

# 匿名内部类

匿名内部类指的是没有名字的内部类，那么对于这种内部类，应该如何去定义，又如何去构造和实例化呢？下面这个例子中展示了其语法细节以及其作用。

```java
/KK.java
import static com.mazhangjing.Print.*;
public class KK {
    public int i;
    KK(int i){ this.i = i; }
    public void get_i(){
        print("i is "+i);
    }
    public static void main(String[] args){
        HH x = new HH();
        KK a = x.getKK(3);
        a.get_i();
        KK b = x.getKK2(3);
        b.get_i();
    }
}
class HH {
    public KK getKK(int n){
        return new KK(n);
    }
    //参数放置在方法参数中进行传递
    public KK getKK2(int n){
    //一个继承自KK普通类的匿名方法，除此之外，还可以继承自接口或者抽象类
        return new KK(n) {
            int i = n * 10;
            public void get_i() {
                print("i is "+i);
            }
        }; //这是个表达式，记得加分号
        //注意，这个匿名类的作用域到这里就没了
    }
}
```

正如 `public KK getKK2(int n)` 静态方法所表现的那样，在这里的内部类就好像是一个存在于方法中的内部类。直接使用 `return new + 类型() 来代替 return new className()` 来调用匿名内部类。而在方法体中，和正常的内部类没有区别。此外，如果需要传递参数，那么参数一般放置在方法参数中，因为这里是方法作用域，内部类可以直接访问，然后像这样 `return new KK(n)` 传递进来即可。

匿名内部类的方法必须是由其所继承或者实现的接口所定义的，而不能是其本身就有的，否则无法调用。而继承或者实现如何体现呢？这里又不能使用 `extends or implements` 关键字，诀窍在于类型，如果返回这个类型是继承或者接口所要求的，那么就可以看作是继承或者接口实现。

## 匿名内部类和工厂模式改进

```java
/Game.java
import java.util.Random; import static com.mazhangjing.Print.*;
public class Games {
    public static void main(String[] args){
        Games x = new Games();
        x.goPlay(Dice8.factory());
    }
    public void goPlay(GameFactory gf){
        Game g = gf.getGame();
        g.play();
    }
}
interface Game {
    void play();
}
interface GameFactory {
    Game getGame();
}
class Dice8 implements Game {
    private Random rand = new Random();
    public void play() {
        int result = rand.nextInt(7)+1;
        print("Welcome to play DICE8\n" +"You get "+ result);
    }
    public static GameFactory factory(){
        return new GameFactory() {
            public Game getGame() {
                return new Dice8();
            }
        };
    }
}
```

对于之前而言，我们需要在客户端放置工厂类的一个实例，如果嫌麻烦，那么可以直接在每个产品内部提供一个静态的匿名方法，这样通过调用Dice8.factory()即可返回一个对象，这种改进进一步提高了灵活性，现在彻底不需要在客户端（用于组合的类）中构造任何实例了，静态匿名内部类允许我们放置一个类似于“常量”的东西进去，如果需要修改，只用更改这个匿名内部类所返回的对象即可。

静态匿名内部类在这里的作用比较有意思，这可以看作是一种非常简洁的简写方式。这就好像一个通电的自动售货机，本来出现的应该是产品，而现在投币下去，你得到的却是另一个通电的自动售货机，你可以在这个新的嵌套的售货机中购买产品。

总的来说，匿名内部类是一种更加灵活和精巧的内部类，可以看作是一种方法作用域内部类的简写，因此可以在外围类的某些地方发挥特殊作用。（比如，对于内部类而言，如果你不需要其对于外围类过多的权限访问，那么将其作为原本返回内部类对象的方法的返回值）。

# 嵌套类

## 用作测试的嵌套类

嵌套类是一种特殊的内部类，其类前含有`static`关键字限定，因此其不能访问外围类中的非静态字段和方法。一般而言，会选择在外围类设置一个方法以返回这个嵌套类，比如 `content()` 在下面的代码中。嵌套类本身可以继续嵌套另一个嵌套类。

```java
/Nest.java
import static com.mazhangjing.Print.*;
public class Nest {
    public static void main(String[] args){
        Nest.content().getInfo();
        Nest.content().soncontent().getInfo();
    }
    
    public static Content content(){return new Content();}
    //对于每个类有xx.class字节码文件
   //但是对于内部类，因为其关系和外围类紧密，因此实际中采用`xx＄SonContent.class`这样表示
    public static class Content {
        void getInfo(){print("Hello from Nest static class Content");}
        public static SonContent soncontent() {return new SonContent();}
        //嵌套类的嵌套类
        public static class SonContent {
            void getInfo(){print("Hello from Nest static class SonContent");}
        }
    }
    /**嵌套类用作class测试：这种写法不用在main中添加额外代码，
     * 需要执行Nest$Tester.class，使用完删除即可*/
    public static class Tester {
        public static void main(String[] args){
            Nest.content().getInfo();
            Nest.content().soncontent().getInfo();
        }
    }
}
```

嵌套类的作用之一是用作单元测试，因为Java会对每个类生成一个 `xxx.class` 的文件。而对于内部类，包括所有需要依赖外部类的类，都是用 `Outter＄Inner.class` 这样表示。因此，写一个静态的内部类（嵌套类），就可以生成一个这样的文件，因为这个文件在 `Nest` 内部，因此可以用作这个类的测试。使用 `java Nest＄Tester.class` 即可。这样和运行 `public static void main()` 中的代码效果一样，只是，如果你有洁癖，不想让最后的产品的二进制带有 main 方法中的测试代码的话，嵌套类也可以这样做。

## 接口内部的嵌套类

```java
import static com.mazhangjing.Print.*;
//本例用来演示嵌套类用作接口内部的公用代码的操作
public interface NestInt {
    int num = 0;
    void methodA();
    void methodB();
    //在接口中声明一个类，这个类自然有了 static 和 final 的属性，也就成为了嵌套类
    //为了进行单元测试，实现接口的方法即可。
    class Test implements NestInt{
        public void methodA() { print("Hello in methodA"); }
        public void methodB() { }
        public static void main(String[] args){
            Test x = new Test();
            x.methodA();
        }
    }
}
```

这个例子展示了嵌套类用作接口的测试工具，其提供了一种接口的实现。或者说，可以将一些接口所有实现都需要共用的东西放到这个嵌套类中。这里真的很神奇，因为接口竟然可以包含这个类以及允许这个类包含所有接口的实现。因为接口内部本身就是 static，因此不用声明，默认就是嵌套类。此外，还是需要 implements 一下接口，这样才能对接口进行测试。在这个嵌套类中写上 main 方法，可以用来测试其自身。这真有趣。Test 被用作测试 NestInt 的某个实现，而 Test.main 被用来测试 Test 的实现。

# 内部类是必须的吗？

## 根据需要选用

```java
import static com.mazhangjing.Print.*;
public class Test {
    public static void main(String[] args){
        A a = new A();
        B b = new B(5);
        for (int i = 0; i < 5; i++){
            b.add(a.getU()); }
        b.go();
    }
}
interface U {
    void methodA();
    void methodB();
    void methodC();
}
class A {
    //典型的由A的内部类实现的接口，相比A亲自实现，其更能说明A和U的关系，毕竟
    //在调用 b.add(a)的时候，很容易被 `is-a` 关系搞糊涂。因为a和b可能一一对应，但是直接传递过去逻辑不通
    //比如：Screen.next(Stimulate) Screen.next(Stimulate.getShow()) 后者关系更清晰一些
    U getU(){
      return new U() {
          public void methodA() { print("methodA");}
          public void methodB() { print("methodB");}
          public void methodC() { print("methodC");}
      };
    }
}
class B {
    B(int n){len = n;list = new U[len];}
    private int len = 0,current = 0;
    private U[] list;
    void add(U u) {
        list[current++] = u; }
    void go() {
        for (int i = 0; i < list.length; i++){
            U x = list[i];
            x.methodA();
            x.methodB();
            x.methodC();
        }
    }
}
```

上面这个例子中，B是一个U接口的列表。A本来可以实现U，但是这里选择了内部类。在Test类中，我们使用A构造了U类型的对象，然后调用B添加到其列表中，然后遍历B并且调用U的三个方法。可以看到，这里耦合的是U和B，那么，为什么B不亲自实现U呢？这样可能造成逻辑混乱，现在B仅仅是一个包含U的列表，有一些对这些列表进行处理的方法，它实现U的话，情理不通。那么，将A作为B的一个内部类呢？这样就好很多。逻辑清楚。

这里之所以把A拿出来的原因是说明，在没有内部类的时候，要解决这个问题，需要创建一个A类，然后实例化其之后，再和B交互。你可以把A看作B的一个内部类就好。

这里的问题是，A为什么不是U的一个实现，而是在其内部写了一个匿名内部类以实现U呢？毕竟其区别仅仅在于 `b.add(a)` `b.add(a.getU())` 这里。

对于这个例子来说，没什么比较。但是如果考虑把U看作技能，A看作超级英雄，B看作超级英雄集合， 那么使用 集合.添加(超级英雄) 更加合理。 但是，如果B 的调用不是为了将 英雄添加到集合，比如，是使用一个技能去救人，那么 集合.释放(超级英雄) 就不合适了。因此，使用内部类完全取决于你是否需要多重继承，以及使用直接实现或者继承能不能很好的完成任务。如果不能，那么就使用另外一种方法。

## 控制框架模式示例

对于以下这个例子，只有 `Event - Controller` 这两个抽像基类，Controller 包含 Event 的列表，并且遍历 Event，调用其方法。我们看到，正是因为选用了在 Controller 的继承 GreenHouseController 类中选择嵌套 Event 的多个子类，因此调用 GHController.new SubEvent() 可以返回多种不同的Event 行为。

对于这个例子，我们看到，Event 肯定和 Controller 是无法通过合理的继承或者接口进行链接的，如果 `Controller extends Event` 那么只能够实现一种Event，如果 `Controller implements EventInterface`，那么也只能保留一种Event的实现方式。因此，作为二道贩子的内部类登场了。将 Event 进行继承的扩展，成为 `WaterOn extends Event, WaterOff..., LightOn..., LightOff...` ，之后，将这些特殊化的类包裹在 `Controller` 内部，这里一个小技巧是，我们同样继承了 `Controller`，因为这些 Event 很具体，所以最好换个皮包裹这些子类，因此有了 `GreenHouseController` 这个东西。

注意，我们看到相比较使用“组合”所带来的优势，组合难以扩展，而 `GreenHouseController` 为单纯的实现，因此继承 `GreenHouseController` 的 ` WATController` 毫不费力的获得了 `GHCer` 的所有子类。

```java
/Event.java
import static com.mazhangjing.Print.*;
/**事件基类，提供延迟时间以初始化时间，当调用ready()并得到状态后，进行action()操作*/
public abstract class Event {
    private long eventtime;
    protected final long delaytime;
    public Event(long delaytime){
        this.delaytime = delaytime;
        start();
    }
    /**设置事件开始的时间*/
    public void start(){
        eventtime = System.nanoTime() + delaytime;
    }
    /**独立的原因是，可能有除了时间以外的因素控制事件开始
     * 因此作为独立的开关控制*/
    public boolean ready(){
        return System.nanoTime() >= eventtime;
    }
    /**需要子类提供具体执行命令，在action中调用start可重复进行*/
    public abstract void action();
}
/Controller.java
import java.util.ArrayList;
import java.util.List;
/**控制器，包含一个事件列表，addEvent可向列表中添加方法
 * run()则会遍历列表，当发现事件.ready()好了之后，就直接调用.action()执行
 * 执行调用后从列表删除，当列表没有元素的时候结束程序*/
public class Controller {
    private List<Event> eventList = new ArrayList<Event>();
    public void addEvent(Event c){
        eventList.add(c); }
    public void run() {
        while (eventList.size() > 0){
            for (Event e: new ArrayList<Event>(eventList)){
                if (e.ready()){
                    System.out.println(e);
                    e.action();
                    eventList.remove(e);
                }
            }
        }
    }
}
/GreenHouseController.java
public class GreenHouseController extends Controller {
    private boolean light = false;
    /**继承自Event的事件，重载了action和toString，注意
     * 这里的内部类和主类耦合，LightOn操纵着light的值。*/
    public class LightOn extends Event{
        public LightOn(long delaytime){super(delaytime);}
        public void action() { light = true; }
        public String toString() { return "Light is on"; }
    }
    public class LightOff extends Event{
        public LightOff(long delaytime){super(delaytime);}
        public void action() { light = false; }
        public String toString() { return "Light is off"; }
    }
    private boolean fan = false;
    public class FanOn extends Event {
        public FanOn(long delaytime) { super(delaytime); }
        public void action() { fan = true; }
        public String toString() { return "Fan is on"; }
    }
    public class FanOff extends Event {
        public FanOff(long delaytime) { super(delaytime); }
        public void action() { fan = false; }
        public String toString() { return "Fan is off"; }
    }
    public class Bell extends Event {
        public Bell(long delaytime){super(delaytime);}
        public void action() {
            //和外围类方法交互，使用addEvent添加设置好的事件
            addEvent(new Bell(delaytime));
        }
        public String toString() {
            return "Bing!"; }
    }
    public class Restart extends Event {
        private Event[] eventList;
        public Restart(long delaytime, Event[] eventList) {
            super(delaytime);
            this.eventList = eventList;
            for (Event e: eventList){
                addEvent(e); }
        }
        public void action() {
            for (Event e : eventList){
                e.start();
                addEvent(e); }
            start();
            addEvent(this);
        }
        public String toString() {
            return "Restarting System...";
        }
    }
    public static class Terminate extends Event {
        public Terminate(long delaytime) {super(delaytime);}
        public void action() {System.exit(0);}
        public String toString() {
            return "Terminating...";
        }
    }
    public static void main(String[] args){
        GreenHouseController gc = new GreenHouseController();
        gc.addEvent(gc.new Bell(0));
        Event[] eventList = {
                gc.new LightOn(200),
                gc.new LightOff(400),
                gc.new FanOn(200),
                gc.new FanOff(400),
        };
        //当调用Restart进行构造的时候，外围类中已经添加了这些event。而在Restart中依然保留了一份
        gc.addEvent(gc.new Restart(2000,eventList));
        //这就相当于把那些事件添加到外围类后又添加了自己
        //当执行的时候，调用action方法，重新添加了所有eventList和自身
        //gc.addEvent(new GreenHouseController.Terminate(40000));
        gc.run();
    }
}
/WATController.java
class WATController extends GreenHouseController {
    private boolean fan = false;
    public class FanOff extends Event {
        public FanOff(long delaytime) { super(delaytime); }
        public void action() { fan = false; }
        public String toString() { return "Fan is off by extends ..."; }
    }
    public static void main(String[] args){
        WATController gc = new WATController();
        gc.addEvent(gc.new Bell(0));
        Event[] eventList = {
                gc.new LightOn(200),
                gc.new LightOff(400),
                gc.new FanOn(200),
                gc.new FanOff(400),
        };
        //当调用Restart进行构造的时候，外围类中已经添加了这些event。而在Restart中依然保留了一份
        gc.addEvent(gc.new Restart(2000,eventList));
        //这就相当于把那些事件添加到外围类后又添加了自己
        //当执行的时候，调用action方法，重新添加了所有eventList和自身
        //gc.addEvent(new GreenHouseController.Terminate(40000));
        gc.run();
    }
}

```

## 用作闭包和回调

内部类可用作闭包。闭包指的是一个对象，这个对象保有主对象的一些信息，可以返回主对象。对于继承的工厂模式而言，使用闭包可以解决内部无限循环的问题。这个问题在RTTI中使用类字面量也可解决，并且更优雅。

```java
interface Factory<T> { T create();}

class Part {
    private static Random rand = new Random();
    static List<Factory<? extends Part>> partfactories =
            new ArrayList<>();
    static {
        partfactories.add(new FueFilter.Callback());
        partfactories.add(new OilFilter.Callback());
        partfactories.add(new FanBelt.Callback());
    }
    public static Part createRandom(){
        int n = rand.nextInt(partfactories.size());
        return partfactories.get(n).create();
    }
    public String toString() {
        return this.getClass().getSimpleName();
    }
}

/**由于我们想要所有继承自基类的工厂都实现createRandom方法，因此这个在基类中实现。
 * 但是，这个方法需要保存一个列表，而如果我们使用了继承的工厂，那么这个列表就必须出现
 * 在每个工厂中。因此，我们不能对工厂使用Factory接口，因为，一旦如此，那么这个列表
 * 就有可能保存工厂对象本身，那么就会出现无穷递归。因此，这里的列表使用回调/内部类，
 * 而这个回调实现接口，这个回调是静态的，因此很容易就得到工厂产品。
 *
 * 同时现在我们可以获得在每个类中继承的createRandom的能力。虽然这是以清晰的逻辑为代价的。
 * */
class Filter extends Part {}

class FueFilter extends Filter {
    //Factory是一个用于闭包的内部类，暂时保存这个对象而不创建FueFilter对象
    public static class Callback implements Factory<FueFilter> {
        public FueFilter create() {
            return new FueFilter();
        }
    }
}
```

# 内部类使用的细节

## 内部类的继承

如下代码所示，如果需要继承内部类的时候，因为内部类有外部类的引用，并且依赖外部类，因此需要使用 `extends Outer.Inner` 这样的声明，此外，对于初始化，需要传入 `OuterInstance` 这个外围类的实例用以 `OuterInstance.super()` 方法进行外围类的初始化后方可使用。

```java
/**在这个例子中，我们调用了一个内部类用作继承，但是因为内部类需要引用外部类
 * 所以在我们新构造这个类中需要显式的初始化这个外围类，传递一个外围类对象，然后调用o.super()进行外围类的初始化。*/
public class Winner extends Outer.Inner {
    //Winner() {} error
    Winner(Outer o){
        o.super();
    }
}
class Outter {
    class Inner {}
}
```

## 内部类不可覆盖

内部类覆盖问题一般发生在我继承了一个外围类，然后重载了内部类方法，问现在是谁在起作用。比如上面例子的 `WATController.java` 一样。结论是：内部类不可以覆盖，因为其具有各自的作用域，因此会独立存在。但是，可以显式的继承内部类。

因此，不会出现重载，但是会调用新方法。



——————————

**更新日志**

2018-07-28 1.4 添加闭包相关内容